In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/densenet_pcam_cbam/pytorch/default/1/densenet121_chunked_cbam.pth
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_train_y.h5
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_valid_y.h5
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_valid_meta.csv
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_valid_x.h5
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_train_mask.h5
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_train_meta.csv
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_test_y.h5
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_test_meta.csv
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_test_x.h5
/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_train_x.h5-001/camelyonpatch_level_2_split_train_x.h5
/kaggle/input/densenet_pcam/pytorch/default/1/densenet121_chunked.pth


In [9]:
# Core
import os
import time
import random
import yaml
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

# Metrics
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix
)


In [10]:
import yaml

config = {
    "seed": 1337,
    "n_seeds": 3,
    "device": "cuda:0",
    "deterministic": True,

    "data": {
        "dataset_version": "pcam_v1",
        "splits_file": "splits/patient_level_fold0.json",
        "img_size": 128,
        "normalize": {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]},
        "decode": "pillow_rgb",
        "stain_norm": "none",
        "augment": {
            "train": [
                {"hflip": 0.5},
                {"vflip": 0.5},
                {"color_jitter": {"b": 0.2, "c": 0.2, "s": 0.2}},
                {"blur": {"p": 0.2, "k": 3}},
            ],
            "val": []
        },
        "sampler": "random",
        "samples_per_epoch": None,
    },

    "loader": {
        "batch_size": 64,
        "grad_accum": 1,
        "num_workers": 4,
        "pin_memory": True,
        "persistent_workers": True,
    },

    "model": {
        "backbone": "densenet121",
        "weights": "imagenet_v1",
        "precision": "fp32",
        "dropout": 0.0,
        "freeze_backbone": False,
    },

    "objective": {
        "loss": "bce_logits",
        "pos_weight": None,
        "label_smoothing": 0.0,
        "focal": {"enable": False, "alpha": 0.25, "gamma": 2.0},
        "clip_grad_norm": None,
        "calibration": {"enable": False},
        "decision_threshold": 0.5,
        "tta": {"enable": False},
    },

    "optim": {
        "name": "adamw",
        "lr": 3.0e-4,
        "weight_decay": 1.0e-4,
        "betas": [0.9, 0.999],
        "schedule": "cosine",
        "t_max": 30,
        "warmup_steps": 0,
        "epochs": 10,
        "ema": {"enable": False, "decay": 0.999},
    },

    "eval": {
        "metrics": ["roc_auc", "pr_auc", "acc", "f1"],
        "ci_bootstrap": 2000,
        "select_best_by": "roc_auc",
        "eval_every": "epoch",
    },

    "env": {
        "torch": "2.3.1",
        "torchvision": "0.18.1",
        "cuda": "12.1",
        "cudnn_deterministic": True,
        "cublas_workspace": ":4096:8",
    }
}

# Save to YAML
with open("experiment.yaml", "w") as f:
    yaml.dump(config, f, sort_keys=False)

print("✅ experiment.yaml file created")

✅ experiment.yaml file created


In [11]:
with open("experiment.yaml", "r") as f:
    cfg = yaml.safe_load(f)


device = torch.device(cfg["device"] if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda:0


In [12]:
# -----------------------------
# Load Trained Model
# -----------------------------
cfg_model = cfg["model"]

# Recreate model architecture
if cfg_model["backbone"] == "densenet121":
    model = models.densenet121(weights=None)  # No pretrained weights needed for inference
    model.classifier = nn.Sequential(
        nn.Dropout(cfg_model["dropout"]),
        nn.Linear(model.classifier.in_features, 1)
    )
else:
    raise NotImplementedError(f"Backbone {cfg_model['backbone']} not implemented")

# Load trained weights
model_path = "/kaggle/input/densenet_pcam/pytorch/default/1/densenet121_chunked.pth"
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"✅ Model loaded from {model_path}")
except FileNotFoundError:
    print(f"❌ Model file {model_path} not found. Please train the model first.")
    exit()

model = model.to(device)
model.eval()

✅ Model loaded from /kaggle/input/densenet_pcam/pytorch/default/1/densenet121_chunked.pth


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [13]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from PIL import Image

# --- Grad-CAM helper ---
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer

        self.gradients = None
        self.activations = None

        # Hook to get gradients and activations
        def forward_hook(module, input, output):
            self.activations = output.detach()
        
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0].detach()
        
        target_layer.register_forward_hook(forward_hook)
        target_layer.register_backward_hook(backward_hook)

    def generate(self, x, target_class=None):
        # Forward pass
        out = self.model(x)

        if target_class is None:
            target_class = out.argmax(dim=1).item()
        
        # Backward pass
        self.model.zero_grad()
        class_score = out[:, target_class]
        class_score.backward(retain_graph=True)

        # Grad-CAM
        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        gradcam_map = (weights * self.activations).sum(dim=1, keepdim=True)
        gradcam_map = F.relu(gradcam_map)

        # Normalize between 0 and 1
        gradcam_map = F.interpolate(gradcam_map, size=x.shape[2:], mode='bilinear', align_corners=False)
        gradcam_map = gradcam_map.squeeze().cpu().numpy()
        gradcam_map = (gradcam_map - gradcam_map.min()) / (gradcam_map.max() - gradcam_map.min() + 1e-8)

        return gradcam_map


In [14]:
import random
import matplotlib.pyplot as plt
import numpy as np

# 1. Find indices for each class in the test dataset
all_labels_array = np.array([test_dataset[i][1] for i in range(len(test_dataset))])
tumor_indices = np.where(all_labels_array == 1)[0]
normal_indices = np.where(all_labels_array == 0)[0]

# 2. Randomly select 5 from each
selected_tumor = np.random.choice(tumor_indices, 5, replace=False)
selected_normal = np.random.choice(normal_indices, 5, replace=False)

selected_indices = np.concatenate([selected_tumor, selected_normal])
class_map = {0: "Normal", 1: "Tumor"}

# 3. Generate Grad-CAM for each selected image
for idx in selected_indices:
    img, label = test_dataset[idx]
    input_tensor = img.unsqueeze(0).to(device)

    # Forward pass and Grad-CAM
    output = model(input_tensor)
    prob = torch.sigmoid(output).item()
    pred_label = int(prob > 0.5)

    heatmap = gradcam.generate(input_tensor)

    # Convert image to numpy for plotting
    img_np = np.transpose(img.cpu().numpy(), (1, 2, 0))

    # Plot image + heatmap
    plt.figure(figsize=(4,4))
    plt.imshow(img_np)
    plt.imshow(heatmap, cmap='jet', alpha=0.5)
    plt.axis('off')
    plt.title(f"Pred: {pred_label} ({class_map[pred_label]}), Prob: {prob:.2f}, True: {label} ({class_map[label]})")
    plt.show()

    # Print descriptive message
    if pred_label == label:
        print(f"The model correctly predicts this is {class_map[pred_label]}.\n")
    else:
        print(f"The model predicts this is {class_map[pred_label]}, but it is actually {class_map[label]}.\n")


NameError: name 'test_dataset' is not defined

In [ ]:
import yaml

config = {
    "seed": 1337,
    "n_seeds": 3,
    "device": "cuda:0",
    "deterministic": True,

    "data": {
        "dataset_version": "pcam_v1",
        "splits_file": "splits/patient_level_fold0.json",
        "img_size": 128,
        "normalize": {"mean": [0.485, 0.456, 0.406], "std": [0.229, 0.224, 0.225]},
        "decode": "pillow_rgb",
        "stain_norm": "none",
        "augment": {
            "train": [
                {"hflip": 0.5},
                {"vflip": 0.5},
                {"color_jitter": {"b": 0.2, "c": 0.2, "s": 0.2}},
                {"blur": {"p": 0.2, "k": 3}},
            ],
            "val": []
        },
        "sampler": "random",
        "samples_per_epoch": None,
    },

    "loader": {
        "batch_size": 64,
        "grad_accum": 1,
        "num_workers": 4,
        "pin_memory": True,
        "persistent_workers": True,
    },

    "model": {
        "backbone": "densenet121",
        "weights": "imagenet_v1",
        "precision": "fp32",
        "dropout": 0.0,
        "freeze_backbone": False,
        "attention_type": "CBAM",  # Options: "SE", "ECA", "CBAM", "None"
        "attention_params": {
            "SE": {
                "reduction": 16
            },
            "ECA": {
                "k_size": 3
            },
            "CBAM": {
                "reduction": 16,
                "kernel_size": 7
            }
        }
    },

    "objective": {
        "loss": "bce_logits",
        "pos_weight": None,
        "label_smoothing": 0.0,
        "focal": {"enable": False, "alpha": 0.25, "gamma": 2.0},
        "clip_grad_norm": None,
        "calibration": {"enable": False},
        "decision_threshold": 0.5,
        "tta": {"enable": False},
    },

    "optim": {
        "name": "adamw",
        "lr": 3.0e-4,
        "weight_decay": 1.0e-4,
        "betas": [0.9, 0.999],
        "schedule": "cosine",
        "t_max": 30,
        "warmup_steps": 0,
        "epochs": 10,
        "ema": {"enable": False, "decay": 0.999},
    },

    "eval": {
        "metrics": ["roc_auc", "pr_auc", "acc", "f1"],
        "ci_bootstrap": 2000,
        "select_best_by": "roc_auc",
        "eval_every": "epoch",
    },

    "env": {
        "torch": "2.3.1",
        "torchvision": "0.18.1",
        "cuda": "12.1",
        "cudnn_deterministic": True,
        "cublas_workspace": ":4096:8",
    }
}

# Save to YAML
with open("experiment.yaml", "w") as f:
    yaml.dump(config, f, sort_keys=False)

print("✅ experiment_cbam.yaml file created")
with open("experiment.yaml", "r") as f:
    cfg2 = yaml.safe_load(f)


In [ ]:
# Squeeze-and-Excitation Module
class SqueezeExcitation(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SqueezeExcitation, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(channels, channels // reduction, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction, channels, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).squeeze(-1).squeeze(-1)
        y = self.fc1(y)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.sigmoid(y).unsqueeze(-1).unsqueeze(-1)
        return x * y.expand_as(x)



# CBAM Module
class ChannelAttention(nn.Module):
    def __init__(self, channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x).squeeze(3).squeeze(2))
        max_out = self.fc(self.max_pool(x).squeeze(3).squeeze(2))
        out = avg_out + max_out
        return self.sigmoid(out).unsqueeze(2).unsqueeze(3)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(out)
        return self.sigmoid(out)

class CBAM(nn.Module):
    def __init__(self, channels, reduction=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(channels, reduction)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        out = x * self.ca(x)
        out = out * self.sa(out)
        return out

In [ ]:
from torchvision.models.densenet import _DenseBlock
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import cv2
import h5py

# -----------------------------
# Load CBAM DenseNet121
# -----------------------------
cfg2_model = cfg2["model"]

if cfg2_model["backbone"] == "densenet121":
    base_model = models.densenet121(weights=None)

    # Re-insert attention layers after every DenseBlock
    features_children = list(base_model.features.children())
    new_modules = []
    se_channels = [256, 512, 1024, 1024]
    attn_count = 0

    for module in features_children:
        new_modules.append(module)
        if isinstance(module, _DenseBlock):
            attn_type = cfg2_model["attention_type"]
            if attn_type == "SE":
                attn = SqueezeExcitation(se_channels[attn_count], **cfg2_model["attention_params"]["SE"])
            elif attn_type == "ECA":
                attn = ECA(se_channels[attn_count], **cfg2_model["attention_params"]["ECA"])
            elif attn_type == "CBAM":
                attn = CBAM(se_channels[attn_count], **cfg2_model["attention_params"]["CBAM"])
            else:
                attn = None
            if attn is not None:
                new_modules.append(attn)
            attn_count += 1

    base_model.features = nn.Sequential(*new_modules)

    # Binary classifier
    base_model.classifier = nn.Sequential(
        nn.Dropout(cfg2_model["dropout"]),
        nn.Linear(base_model.classifier.in_features, 1)
    )
else:
    raise NotImplementedError(f"Backbone {cfg2_model['backbone']} not implemented")

# -----------------------------
# Load trained weights
# -----------------------------
model_path = "/kaggle/input/densenet_pcam_cbam/pytorch/default/1/densenet121_chunked_cbam.pth"
model_cbam = base_model
state_dict = torch.load(model_path, map_location=device)
model_cbam.load_state_dict(state_dict)
model_cbam = model_cbam.to(device)
model_cbam.eval()
print("✅ CBAM model loaded and ready")

# -----------------------------
# Load train images and labels (masks not needed here)
# -----------------------------
train_x_path = "/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_train_x.h5"
train_y_path = "/kaggle/input/pcamv1/pcamv1/camelyonpatch_level_2_split_train_y.h5"

with h5py.File(train_x_path, "r") as x_file, \
     h5py.File(train_y_path, "r") as y_file:

    X_train = x_file["x"][:]        # (N, H, W, C)
    Y_train = y_file["y"][:]        # (N,)

print(f"✅ Loaded train images: {X_train.shape}, labels: {Y_train.shape}")

# -----------------------------
# Grad-CAM setup
# -----------------------------
target_layer_cbam = model_cbam.features[-1]
gradcam_cbam = GradCAM(model_cbam, target_layer_cbam)

class_map = {0: "Normal", 1: "Tumor"}
mean = np.array(cfg2["data"]["normalize"]["mean"]).reshape(1,1,3)
std  = np.array(cfg2["data"]["normalize"]["std"]).reshape(1,1,3)

# -----------------------------
# Visualization loop (heatmap only)
# -----------------------------
for idx in selected_indices:
    img = torch.tensor(X_train[idx].transpose(2,0,1), dtype=torch.float32)  # (C,H,W)
    label = int(Y_train[idx])

    input_tensor = img.unsqueeze(0).to(device)

    # Forward pass
    with torch.no_grad():
        output = model_cbam(input_tensor)
    prob = torch.sigmoid(output).item()
    pred_label = int(prob > 0.5)

    # Grad-CAM
    heatmap = gradcam_cbam.generate(input_tensor)
    if isinstance(heatmap, torch.Tensor):
        heatmap = heatmap.detach().squeeze().cpu().numpy()
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)

    # Resize heatmap to image size
    H, W = img.shape[1], img.shape[2]
    heatmap_resized = cv2.resize(heatmap, (W,H), in_
